In [ ]:
#Installing libraries
!pip install transformers==3.0.2
!pip install nlp==0.4.0
!pip install pyarrow==0.16.0
!pip install sentencepiece==0.1.96

In [ ]:
import torch
import nlp
from tqdm import tqdm
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration

In [ ]:
#Befere running evaluation we have to convert tensorflow checkpoint into pytorch model.
#See here: https://github.com/huggingface/transformers/blob/master/src/transformers/convert_t5_original_tf_checkpoint_to_pytorch.py

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Import the tokenizer and the config file from drive
#The config file can be download from this link: https://s3.amazonaws.com/models.huggingface.co/bert/t5-small-config.json

spm_path = '/content/drive/MyDrive/T5-Extension/Tokenizer/dl4se.model'
config_file = '/content/drive/MyDrive/T5-Extension/Trained-Models/with-pretraining-new/MT-Task-Balanced/Pytorch-Model/config.json'
config = T5Config.from_json_file(config_file)
tokenizer = T5Tokenizer.from_pretrained(spm_path)

In [ ]:
!cp /content/drive/MyDrive/T5-Extension/Miscellaneous/bfp_dataset_script.py /content/bfp_dataset_script.py
#!cp /content/drive/MyDrive/T5-Extension/Miscellaneous/bfp_dataset_script.py /content/assert_dataset_script.py

In [ ]:
# Change the prefix when want to evaluate different tasks
# (1) generate small patch
# (2) generate medium patch
# (3) generate abt assert
# (4) generate raw assert

# If you're evaluating abt/raw assert generative tasks, change example['method'].lower() for the input_text and  example['assertion'].lower() for the target_text

def add_eos_to_examples(example):
    example['input_text'] = 'generate small patch: %s </s>' % example['buggy']#.lower()
    example['target_text'] = '%s </s>' % example['fixed']#.lower()
    return example


def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input_text'], pad_to_max_length=True, max_length=512)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target_text'], pad_to_max_length=True, max_length=512)

    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'target_ids': target_encodings['input_ids'],
        'target_attention_mask': target_encodings['attention_mask']
    }

    return encodings

In [ ]:
#!cp '/content/drive/MyDrive/T5-Extension/Datasets/tsv/fine-tuning/BFmedium/test.tsv' test.tsv
#!cp '/content/drive/MyDrive/T5-Extension/Datasets/tsv/fine-tuning/AGraw/test.tsv' test.tsv
#!cp '/content/drive/MyDrive/T5-Extension/datasets/tsv/fine-tuning/CS/test.tsv' test.tsv
#!cp '/content/drive/MyDrive/T5-Extension/datasets/tsv/fine-tuning/MG/test.tsv' test.tsv
#!cp '/content/drive/MyDrive/T5-Extension/Datasets/tsv/fine-tuning/AGabs/test.tsv' test.tsv
#!cp '/content/drive/MyDrive/T5-Extension/Datasets/tsv/fine-tuning/BFsmall/test.tsv' test.tsv


In [ ]:
%%capture

# Here we have to change the script for loading the dataset
# Pick the script according to the task and load it on this colab instance
# Make sure to load the test set as well; otherwise, it doesn't work.

valid_dataset = nlp.load_dataset('/content/bfp_dataset_script.py', split=nlp.Split.TEST)


# map add_eos_to_examples function to the dataset example wise 
valid_dataset = valid_dataset.map(add_eos_to_examples, load_from_cache_file=False)

# map convert_to_features batch wise
valid_dataset = valid_dataset.map(convert_to_features, batched=True, load_from_cache_file=False)


columns = ['input_ids', 'target_ids', 'attention_mask','target_attention_mask']
valid_dataset.set_format(type='torch', columns=columns)


In [ ]:
#The BATCH_SIZE must be set according to the available VRAM.

BATCH_SIZE = 8
dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE)

In [ ]:
#Let's import the ground truth from the test dataset
import pandas as pd

df = pd.read_csv('test.tsv',header=None,sep='\t')

references=[]

for item in df[1]:
  references.append(item)

inputs=[]

for item in df[0]:
  inputs.append(item)

inputs[0], references[0]

In [ ]:
#Set CUDA device to leverage GPU computation
CUDA = torch.device("cuda")

finetuned_model_path = '/content/drive/MyDrive/T5-Extension/Trained-Models/with-pretraining-new/MT-Task-Balanced/Pytorch-Model/pytorch_model.bin'

model = T5ForConditionalGeneration.from_pretrained(
        finetuned_model_path,
        config=config
        ).to(CUDA)
        
model.eval()

In [ ]:
# Change the max_length in model.generate according to specific tasks
# For bfp_small and bfp_medium we set respectively 128 and 256.
# For both abt assert and raw assert tasks, we used 512 as max length


from tqdm import tqdm

predictions = []

BEAM_SIZE = 25

torch.cuda.empty_cache()

for batch in tqdm(dataloader):

      outs = model.generate(
                          input_ids=batch['input_ids'].to(CUDA),
                          attention_mask=batch['attention_mask'].to(CUDA),
                          num_beams=BEAM_SIZE, 
                          max_length=128,
                          num_return_sequences=BEAM_SIZE, 
                          early_stopping=True
                          )
    

    
      outs = [tokenizer.decode(ids, skip_special_tokens=True)  for ids in outs]
      predictions.extend(outs)

In [ ]:
pred_refined = []
for pred in predictions:
    if len(pred)>=2:
      if pred[0]=='"':
          pred = pred[1:]
      if pred[-1]=='"':
          pred = pred[:-1]
    pred_refined.append(pred)
    
len(pred_refined),len(predictions)


In [ ]:
counter_pred = 0

mispred_list = []

sanity_check_list = []

idx = 0

len_prediction=(len(pred_refined))


for i in range(0, len_prediction, BEAM_SIZE):

    items_to_analyze = pred_refined[i:i+BEAM_SIZE]
    target_item = ''.join(references[idx].split(' '))
    
    flag_perfect = False

    
    fpred=open('/content/drive/MyDrive/T5-Extension/Results/Predictions/Multi-Task/Balanced/BF-Task/Small/prediction_@{}.txt'.format(BEAM_SIZE),'a+')
    fpred.write('************************************\n')
    fpred.write('[+] input: {}\n'.format(inputs[idx]))
        

    for pred in items_to_analyze:
        
        pred_ref = ''.join(pred.split(' '))
        
        fpred.write('[*] target: {}\n'.format(references[idx]))
        fpred.write('[-] pred:  {}\n\n'.format(pred))

        if pred_ref == target_item and not flag_perfect:
            counter_pred+=1
            sanity_check_list.append(pred)

            with open('/content/drive/MyDrive/T5-Extension/Results/Predictions/Multi-Task/Balanced/BF-Task/Small/perfect_@{}.txt'.format(BEAM_SIZE),'a+') as fwrite:
                fwrite.write('[+] input: {}\n'.format(inputs[idx]))
                fwrite.write('[*] target: {}\n'.format(references[idx]))
                fwrite.write('[-] pred:  {}\n\n'.format(pred))
            
            flag_perfect = True
        
        else:
          mispred_list.append(pred)
      
    fpred.write('************************************\n')
        
    idx += 1

fpred.close()
print('% of perfect predictions: ',(counter_pred/len(references))*100 )
print(counter_pred)